<a href="https://colab.research.google.com/github/skdding3/OPGG-Project/blob/main/OPGG_JSON_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **0. 사전 작업**

In [ ]:
# drive.mount
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 패키지 설치
import pandas as pd
import json
import ast
from pandas.io.json import json_normalize

# google drive에 있는 파일 경로
file_path = "/content/drive/MyDrive/Colab Notebooks/"

# 파일명
file_name = "lol_kr_v1018_soloRank.json"

## **1. 원본 데이터 불러오기**

In [ ]:
# 빈 리스트 생성 -> 필요한 데이터를 받는 역할
data_1 = []
data_2 = []
data_3 = []
data_4 = []

with open(file_path + file_name, "r") as f:
    reader = pd.read_json(f,
                          lines=True, # 하나의 딕셔너리를 하나의 라인으로 인식하여 불러오는 파라미터
                          chunksize=1000 # json 데이터를 chunk 객체로 가져오는 파라미터 (데이터를 하나하나 가져오기엔 너무 무거우니까 러프하게 묶음으로 가져오는 역할)
                          )
    for chunk in reader:
        # 묶음으로 가져온 chunk 객체에서 필요 없는 column을 drop하는 부분 (이렇게 하면 chunk가 가벼워지니까 데이터를 가져올 수 있다!)
        reduced_chunk_1 = chunk.drop(columns=['participantidentities', 'gameversion', 'platformid', 'gamemode', 'mapid', 'gametype', 'teams', 'participants', 'gameduration', 'gamecreation', 'participantextendedstats'])
        reduced_chunk_2 = chunk.drop(columns=['seasonid', 'queueid', 'gameid', 'participantidentities', 'mapid', 'gametype', 'teams', 'participants', 'gameduration', 'gamecreation', 'participantextendedstats'])
        reduced_chunk_3 = chunk.drop(columns=['seasonid', 'queueid', 'gameid', 'participantidentities', 'gameversion', 'platformid', 'gamemode', 'teams', 'participants', 'gameduration', 'gamecreation', 'participantextendedstats'])
        reduced_chunk_4 = chunk.drop(columns=['seasonid', 'queueid', 'gameid', 'participantidentities', 'gameversion', 'platformid', 'gamemode', 'mapid', 'gametype', 'teams', 'participants', 'participantextendedstats'])
        # 가벼워진 데이터를 빈 리스트에 저장
        data_1.append(reduced_chunk_1)
        data_2.append(reduced_chunk_2)
        data_3.append(reduced_chunk_3)
        data_4.append(reduced_chunk_4)

# 리스트로 받은 데이터를 데이터 프레임으로 변환
data_1 = pd.concat(data_1, ignore_index=True)
data_2 = pd.concat(data_2, ignore_index=True)
data_3 = pd.concat(data_3, ignore_index=True)
data_4 = pd.concat(data_4, ignore_index=True)

In [ ]:
# 데이터 총 개수 : 169831

# seasonid     / 경기 시즌                 / 13값 만 있음              / 삭제
# queueid      / 게임 형태                 / 420값 만 있음             / 삭제 
# gameid       / 각 매치의 ID 값           / 거의 고유 값               / 살림
# gameversion  / 패치 정보                 / 10.18.333.8889값 만 있음  / 삭제
# platformid   / 국가 정보                 / KR값 만 있음              / 삭제
# gamemode     / 게임 모드                 / gamemode값 만 있음        / 삭제
# mapid        / 경기 맵                  / 11값 만 있음              / 삭제
# gametype     / 게임 타입                 / MATCHED_GAME값 만 있음    / 삭제
# gameduration / 경기 진행 시간(초)          / 2500개 Value 있음        / 살림
# gamecreation / 캐릭터 선택이 끝난 시점의 시각 / 거의 고유 값               / 삭제

# 이슈 : gameid - 4626085422, 4635510522, 4619222704 가 2개로 겹침. 의미상 말이 안되는데 이에 대한 확인 필요

## **2. 필요한 칼럼 추출 및 저장**

1. participants는 용량이 너무 커서 별도로 저장
2. 나머지는 하나로 몰아서 저장

In [ ]:
# 살린거 : 'gameid', 'participantidentities', 'teams', 'participants', 'gameduration', 'participantextendedstats'
# 필수삭제 : 'seasonid', 'queueid', 'gameversion', 'platformid', 'gamemode', 'mapid', 'gametype', 'gamecreation'
data = []

with open(file_path + file_name, "r") as f:
    reader = pd.read_json(f,
                          lines=True,
                          chunksize=1000,
                          nrows=169831
                          )
    for chunk in reader:
        reduced_chunk = chunk.drop(columns=['participants', 'seasonid', 'queueid', 'gameversion', 'platformid', 'gamemode', 'mapid', 'gametype', 'gamecreation'])
        data.append(reduced_chunk)

data = pd.concat(data, ignore_index=True)

# 결과 확인
data.head()

In [ ]:
# 데이터 내보내기
data.to_csv("/content/drive/MyDrive/Colab Notebooks/data.csv", header=True, index=False)

## **3. Nest Dictionary를 DataFrame으로 변환 - participantidentities**

In [ ]:
# 종합 데이터 불러오기
data_url = "/content/drive/MyDrive/Colab Notebooks/data.csv"
data_raw = pd.read_csv(data_url)
data_raw.head()

In [ ]:
# participantidentities column 변환하기

# 중괄호 제거
for i in range(0, len(data_raw)) :
    data_raw['participantidentities'][i] = data_raw['participantidentities'][i].replace('[','').replace(']','')

# 빈 리스트 생성
part_list = []

# Dictionary로 형 변환
for i in range(0, len(data_raw)) :
    part_list += ast.literal_eval(data_raw['participantidentities'][i])

# Normalization 진행
norm_data = pd.json_normalize(part_list)
norm_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
